In [ ]:
%load_ext autoreload
%autoreload 2

from pathlib import Path
from cellpose import io
import skimage.io
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('dark_background')
import matplotlib as mpl
%matplotlib inline
mpl.rcParams['figure.dpi'] = 300
from cellpose import dynamics, transforms
from scipy import stats


In [ ]:
from aicsimageio.readers import CziReader
import os
basedir = '/home/kcutler/DataDrive/confocal'
dataset = 'Experiment-712' #720 too dim, 733 too thick, 704 too binned,707 good! try 708 next 
reader = CziReader(os.path.join(basedir,dataset+'.czi'))
im = reader.data.squeeze()

In [ ]:
modeldir = '/home/kcutler/DataDrive/confocal/train60/models/cellpose_residual_on_style_on_concatenation_off_omni_train60_2022_04_28_02_39_03.970940_epoch_3999'

In [ ]:
dim = 3
nclasses = dim+2
nchan = 1
omni = 1
rescale = False
diam_mean = 0
from cellpose import models, core
use_GPU = core.use_gpu()
print('>>> GPU activated? %d'%use_GPU)
# use_GPU = 0 # uh oh, 3D models not working for CPU
model = models.CellposeModel(gpu=use_GPU, pretrained_model=modeldir, net_avg=False, 
                             diam_mean=diam_mean, nclasses=nclasses, dim=dim, nchan=nchan)

In [ ]:
mask_threshold = -7 ##############
diam_threshold = 12
net_avg = 0
cluster = 1
verbose = 1

tile = 0
chans = [0,0]
chans = None
compute_masks = 1
slc_crop = slice(None, -1, None)
# slc_crop = slice(100, 120, None)
# slc_crop = slice(13, 53, None) # 733
# slc_crop = slice(50, 100, None) # 704
# slc_crop = slice(15, 52, None) # 707
slc_crop = slice(30, 70, None) # 712
resample=False

imcrop = im[slc_crop]
# imcrop = im[100:120]
# imcrop = im.copy()
# import torch

masks, flows, styles = model.eval(imcrop,channels = chans,rescale=None,mask_threshold=mask_threshold,net_avg=net_avg,
                                  transparency=True,flow_threshold=0.,omni=omni,resample=resample,verbose=verbose,
                                  diam_threshold=diam_threshold,cluster=cluster,tile=0,
                                  compute_masks=compute_masks)
#     else:

In [ ]:
im.shape

In [ ]:
import napari, ncolor
from omnipose.utils import sinebow

In [ ]:
mnc = ncolor.label(masks)
N = np.max(mnc)
viewer = napari.view_image(imcrop, name='cells')
viewer.add_labels(mnc, name='labels',color=sinebow(N+3))

In [ ]:
viewer.add_image(flows[2], name='dist')
viewer.add_image(flows[4], name='bd')
viewer.add_image(flows[0], name='flow')

In [ ]:
viewer = napari.view_image(flows[0], name='cells')


In [ ]:
mnc.max()

In [ ]:
im.shape

In [ ]:
plt.imshow(flows[2][20]>-9)

In [ ]:
import tifffile
stack = tifffile.imread('/home/kcutler/DataDrive/luca/data/tif_stacks/191.tif')

In [ ]:
viewer = napari.view_image(stack, name='cells')


In [ ]:
from aicsimageio import AICSImage
idir = '/home/kcutler/DataDrive/3D_BBBC/BBBC024_v1_c75_highSNR_foreground/image-labels_0015.ics'
img = AICSImage(idir)


In [ ]:
# img.data  # returns 5D TCZYX numpy array
# img.xarray_data  # returns 5D TCZYX xarray data array backed by numpy
# img.dims  # returns a Dimensions object
# img.dims.order  # returns string "TCZYX"
# img.dims.X  # returns size of X dimension
# img.shape  # returns tuple of dimension sizes in TCZYX order
# img.get_image_data("CZYX", T=0)  # returns 4D CZYX numpy array
import napari
viewer = napari.view_image(img.data.squeeze(), name='cells')


In [ ]:
!pip install aicsimageio[bfio] --force --upgrade

In [ ]:
import numpy as np
import tifffile
from cloudvolume import CloudVolume

vol = CloudVolume(
    "s3://open-neurodata/hildebrand/120809zf104/151009", mip=0, use_https=True
)

# load data into numpy array
cutout = vol[1024:1536, 512:1024, 7520:7536]

In [ ]:
viewer = napari.view_image(vol[1024:1536, 512:1024, 7000:7536], name='cells')


In [ ]:
cutout.shape

In [ ]:
vol.shape

In [ ]:
np.array(vol[:])